There must be an automated process to ingest and store the data.

● Trips with similar origin, destination, and time of day should be grouped together.

● Develop a way to obtain the weekly average number of trips for an area, defined by a
bounding box (given by coordinates) or by a region.

● Develop a way to inform the user about the status of the data ingestion without using a
polling solution.

● The solution should be scalable to 100 million entries. It is encouraged to simplify the
data by a data model. Please add proof that the solution is scalable.

● Use a SQL database.

In [176]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import os

In [177]:
analyze_folder = "D://Documentos//Data Science//Personal Projects//Jobcity Challenge//Data"

In [178]:
for file in os.listdir(analyze_folder):
    if file.split(".")[1] == "csv":    
        analytics_df = pd.read_csv(analyze_folder + "//" + file)
    elif file.split(".")[1] == "xlsx":    
        analytics_df = pd.read_excel(analyze_folder + "//" + file)
        
        

In [179]:
analytics_df["Origin_x"] = analytics_df.origin_coord.str.split(expand=True)[1].str.replace("(","").astype(float).round(0)
analytics_df["Origin_y"] = analytics_df.origin_coord.str.split(expand=True)[2].str.replace(")","").astype(float).round(0)

analytics_df["Destination_x"] = analytics_df.destination_coord.str.split(expand=True)[1].str.replace("(","").astype(float).round(0)
analytics_df["Destination_y"] = analytics_df.destination_coord.str.split(expand=True)[2].str.replace(")","").astype(float).round(0)

In [180]:

analytics_df['datetime'] = pd.to_datetime(analytics_df['datetime'])

time_window = "30min"

analytics_df["Time"] = analytics_df['datetime'].dt.round(time_window).dt.strftime('%H:%M')


analytics_df["Week"] = analytics_df['datetime'].dt.isocalendar().week

In [181]:
df_grouped = analytics_df.groupby(["Origin_x","Origin_y","Destination_x","Destination_y","Time"]).agg({"region": "min","origin_coord": "min","destination_coord" : "min","datetime":"min"})

In [185]:
df_grouped_2 = analytics_df.groupby(["Week","region"]).agg({"origin_coord":"count"}).sort_index().reset_index()

,Week,region,origin_coord
0,18,Hamburg,5
1,18,Prague,10
2,18,Turin,8
3,19,Hamburg,6
4,19,Prague,8
5,19,Turin,6
6,20,Hamburg,5
7,20,Prague,8
8,20,Turin,6
9,21,Hamburg,7


In [188]:
#SQL Connection

import pyodbc

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-VBRKFH1\SQLEXPRESS;'
                      'Database=Bar;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()